In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime
import warnings
warnings.filterwarnings('ignore')

### Extract CSVs into DataFrames

In [2]:

public_covid_file = "Data/public_covid19_canada.csv"
public_covid_canada_df = pd.read_csv(public_covid_file)
public_covid_canada_df.head()

,case_id,provincial_case_id,age,sex,health_region,province,country,date_report,report_week,travel_yn,travel_history_country,locally_acquired
0,1.0,1.0,50-59,Male,Toronto,Ontario,Canada,25-01-2020,19-01-2020,1,China,NaN
1,2.0,2.0,50-59,Female,Toronto,Ontario,Canada,27-01-2020,26-01-2020,1,China,NaN
2,3.0,1.0,40-49,Male,Vancouver Coastal,BC,Canada,28-01-2020,26-01-2020,1,China,NaN
3,4.0,3.0,20-29,Female,Middlesex-London,Ontario,Canada,31-01-2020,26-01-2020,1,China,NaN
4,5.0,2.0,50-59,Female,Vancouver Coastal,BC,Canada,04-02-2020,02-02-2020,0,NaN,Close Contact


In [3]:
mortality_file = "Data/Mortality_Canada.csv"
mortality_canada_df = pd.read_csv(mortality_file)
mortality_canada_df.head()

,death_id,province_death_id,case_id,age,sex,health_region,province,country,date_death_report
0,1,1,60.0,80-89,Male,Vancouver Coastal,BC,Canada,08-03-2020
1,2,1,477.0,70-79,Male,Simcoe Muskoka,Ontario,Canada,11-03-2020
2,3,2,NaN,NaN,NaN,Vancouver Coastal,BC,Canada,16-03-2020
3,4,3,NaN,NaN,NaN,Vancouver Coastal,BC,Canada,16-03-2020
4,5,4,NaN,NaN,NaN,Vancouver Coastal,BC,Canada,16-03-2020


In [4]:
testing_file = "Data/Testing_Canada.csv"
testcases_canada_df = pd.read_csv(testing_file)
testcases_canada_df.head()

,date_testing,province,cumulative_testing,province_source
0,15-03-2020,Alberta,7108,Alberta
1,16-03-2020,Alberta,10598,BC
2,17-03-2020,Alberta,12355,Manitoba
3,18-03-2020,Alberta,14566,New Brunswick
4,19-03-2020,Alberta,17013,NL


In [5]:
recovered_file = "Data/Recovered_Canada.csv"
recovered_canada_df = pd.read_csv(recovered_file)
recovered_canada_df.head()

,date_recovered,province,cumulative_recovered,province_source
0,12-02-2020,Alberta,NaN,Alberta
1,13-02-2020,Alberta,NaN,BC
2,14-02-2020,Alberta,NaN,Manitoba
3,15-02-2020,Alberta,NaN,New Brunswick
4,16-02-2020,Alberta,NaN,NL


### Transform DataFrame

#### Data Cleansing

In [6]:
# Create a filtered dataframe from specific columns & Rename the column headers

mortality_df = mortality_canada_df[['province','date_death_report','age', 'sex',]]
# mortality_df.head()
mortality_df.columns = ['province', 'date', 'age', 'sex']
# mortality_df.head()
#
recovered_df = recovered_canada_df[['date_recovered', 'province', 'cumulative_recovered']]
recovered_df.columns = ['date', 'province', 'cumulative_recovered']
# recovered_df.head()

testcases_df = testcases_canada_df[['date_testing', 'province', 'cumulative_testing']]
testcases_df.columns = ['date', 'province', 'cumulative_testing']   
# testcases_df.head()

confirmed_cases_df = public_covid_canada_df[['provincial_case_id', 'age', 'sex', 'province', 'date_report', 'travel_yn',
       'travel_history_country', 'locally_acquired']]
confirmed_cases_df.columns = ['positive_cases', 'age', 'sex', 'province', 'date', 'travel_yn',
       'travel_history_country', 'locally_acquired']   
# confirmed_cases_df.head()

In [7]:
# Clean the data by dropping no data rows 
mortality_df.dropna(how='all', inplace=True)
recovered_df.dropna(how='any', inplace=True)
testcases_df.dropna(how='all', inplace=True)
confirmed_cases_df.dropna(how='all', inplace=True)

In [8]:
recovered_df = recovered_df[recovered_df['cumulative_recovered'] > 0]

In [9]:
recovered_df.dtypes

date                     object
province                 object
cumulative_recovered    float64
dtype: object

In [10]:
# transforming date field from object to date
recovered_df['date'] = pd.to_datetime(recovered_df['date'], format='%d-%m-%Y')
testcases_df['date'] = pd.to_datetime(testcases_df['date'], format='%d-%m-%Y')
mortality_df['date'] = pd.to_datetime(mortality_df['date'], format='%d-%m-%Y')
confirmed_cases_df['date'] = pd.to_datetime(confirmed_cases_df['date'], format='%d-%m-%Y')

In [11]:
# transforming cumulative_recovered field to int data type
recovered_df = recovered_df.dropna(subset=['cumulative_recovered'])
recovered_df['cumulative_recovered'] = recovered_df['cumulative_recovered'].astype(int)

In [12]:
# check to verify all the fields are okay...
recovered_df.dtypes

date                    datetime64[ns]
province                        object
cumulative_recovered             int64
dtype: object

In [13]:
# checking testcases_df dataframe
testcases_df.dtypes

date                  datetime64[ns]
province                      object
cumulative_testing            object
dtype: object

In [14]:
testcases_df["cumulative_testing"].unique()

array(['7108', '10598', '12355', '14566', '17013', '20360', '23742',
       '26999', '30058', '32776', '35508', '6326', '17912', '21296',
       '26681', '31739', '400', '2280', '2912', '3270', '3534', '3801',
       '4245', '4520', nan, '203', '204', '381', '421', '520', '788',
       '858', '1096', '1255', '1550', '161', '494', '791', '901', '970',
       '938*', '1131', '1336', '418', '676', '941', '1153', '1387',
       '1561', '1847', '2116', '2349', '2525', '2840', '8465', '10178',
       '11171', '13897', '16650', '19511', '23384', '26420', '28506',
       '32457', '35635', '78', '147', '183', '213', '240', '325', '393',
       '416', '550', '6202', '7801', '8934', '10451', '10222*', '10935',
       '12068', '13727', '15763', '30971', '796', '1107', '1978', '2561',
       '3093', '3917', '4536', '5269', '5757', '6270', '153', '222',
       '275', '299', '370', '410', '492', '533', '108', '154', '166',
       '101', '398', '517'], dtype=object)

In [15]:
# Noticed cumulative_testing field is having * with value 
print(testcases_df[testcases_df['cumulative_testing'].isin(['938*','10222*'])])
print("-----")
print(testcases_df[testcases_df['province'].isin(['NL','Quebec'])])

         date province cumulative_testing
52 2020-03-23       NL               938*
93 2020-03-20   Quebec             10222*
-----
         date province cumulative_testing
44 2020-03-15       NL                161
45 2020-03-16       NL                161
46 2020-03-17       NL                494
47 2020-03-18       NL                494
48 2020-03-19       NL                494
49 2020-03-20       NL                791
50 2020-03-21       NL                901
51 2020-03-22       NL                970
52 2020-03-23       NL               938*
53 2020-03-24       NL               1131
54 2020-03-25       NL               1336
88 2020-03-15   Quebec                NaN
89 2020-03-16   Quebec               6202
90 2020-03-17   Quebec               7801
91 2020-03-18   Quebec               8934
92 2020-03-19   Quebec              10451
93 2020-03-20   Quebec             10222*
94 2020-03-21   Quebec              10935
95 2020-03-22   Quebec              12068
96 2020-03-23   Quebec      

In [16]:
# removing wrong data
testcases_df = testcases_df[~testcases_df['cumulative_testing'].isin(['938*','10222*'])]

In [17]:
# transforming cumulative_testing field to int data type
testcases_df = testcases_df.dropna(subset=['cumulative_testing'])
testcases_df['cumulative_testing'] = testcases_df['cumulative_testing'].astype(int)

In [18]:
testcases_df.head()

,date,province,cumulative_testing
0,2020-03-15,Alberta,7108
1,2020-03-16,Alberta,10598
2,2020-03-17,Alberta,12355
3,2020-03-18,Alberta,14566
4,2020-03-19,Alberta,17013


In [19]:
# date field already changed and mortality_df dataframe is ready for next stage...
mortality_df.dtypes

province            object
date        datetime64[ns]
age                 object
sex                 object
dtype: object

In [20]:
confirmed_cases_df.dtypes

positive_cases                   float64
age                               object
sex                               object
province                          object
date                      datetime64[ns]
travel_yn                         object
travel_history_country            object
locally_acquired                  object
dtype: object

In [21]:
confirmed_cases_df['positive_cases'] = confirmed_cases_df['positive_cases'].astype(int)

In [22]:
confirmed_cases_df.dtypes

positive_cases                     int64
age                               object
sex                               object
province                          object
date                      datetime64[ns]
travel_yn                         object
travel_history_country            object
locally_acquired                  object
dtype: object

In [23]:
# Set index for all dataframes
mortality_df.reset_index(drop=True)
recovered_df.reset_index(drop=True)
testcases_df.reset_index(drop=True)
confirmed_cases_df.reset_index(drop=True)

,positive_cases,age,sex,province,date,travel_yn,travel_history_country,locally_acquired
0,1,50-59,Male,Ontario,2020-01-25,1,China,NaN
1,2,50-59,Female,Ontario,2020-01-27,1,China,NaN
2,1,40-49,Male,BC,2020-01-28,1,China,NaN
3,3,20-29,Female,Ontario,2020-01-31,1,China,NaN
4,2,50-59,Female,BC,2020-02-04,0,NaN,Close Contact
...,...,...,...,...,...,...,...,...
3404,655,Not Reported,Not Reported,BC,2020-03-25,Not Reported,NaN,NaN
3405,656,Not Reported,Not Reported,BC,2020-03-25,Not Reported,NaN,NaN
3406,657,Not Reported,Not Reported,BC,2020-03-25,Not Reported,NaN,NaN
3407,658,Not Reported,Not Reported,BC,2020-03-25,Not Reported,NaN,NaN


In [24]:
# adding index column name
mortality_df.index.name = 'id'
recovered_df.index.name = 'id'
testcases_df.index.name = 'id'
confirmed_cases_df.index.name = 'id'

#### Data Grouping

In [25]:
mortality_df2 = mortality_df[['province','date']]
mortality_df2.head()

,province,date
id,,
0,BC,2020-03-08
1,Ontario,2020-03-11
2,BC,2020-03-16
3,BC,2020-03-16
4,BC,2020-03-16


In [26]:
# mortality_grp["month"] = mortality_grp["date"].dt.month
mortality_df2['weekno'] = mortality_df2['date'].dt.week
mortality_df2.head()

,province,date,weekno
id,,,
0,BC,2020-03-08,10
1,Ontario,2020-03-11,11
2,BC,2020-03-16,12
3,BC,2020-03-16,12
4,BC,2020-03-16,12


In [27]:
# mortality_month = mortality_grp.groupby(['month','province','sex','age']).count()
# mortality_df2.fillna('N/A', inplace=True)  -- required only for sex/age grouping
# mortality_month = mortality_grp.groupby(['weekno','province','sex','age']).count()
mortality_grp = mortality_df2.groupby(['weekno','province']).count()
mortality_grp.columns = ["death_cases"]
mortality_grp.head()

death_cases
weekno province             
10     BC                  1
11     Ontario             1
12     Alberta             1
       BC                  9
       Ontario             5

In [28]:
testcases_df.head()

,date,province,cumulative_testing
id,,,
0,2020-03-15,Alberta,7108
1,2020-03-16,Alberta,10598
2,2020-03-17,Alberta,12355
3,2020-03-18,Alberta,14566
4,2020-03-19,Alberta,17013


In [29]:
testcases_df.columns

Index(['date', 'province', 'cumulative_testing'], dtype='object')

In [30]:
testcases_df2 = testcases_df[['date', 'province', 'cumulative_testing']]
testcases_df2['weekno'] = testcases_df2['date'].dt.week
testcases_df2.fillna('N/A', inplace=True)
testcases_sorted = testcases_df2.sort_values(["province","date"])
testcases_sorted.head()

,date,province,cumulative_testing,weekno
id,,,,
0,2020-03-15,Alberta,7108,11
1,2020-03-16,Alberta,10598,12
2,2020-03-17,Alberta,12355,12
3,2020-03-18,Alberta,14566,12
4,2020-03-19,Alberta,17013,12


In [31]:
testcases_df.head()

,date,province,cumulative_testing
id,,,
0,2020-03-15,Alberta,7108
1,2020-03-16,Alberta,10598
2,2020-03-17,Alberta,12355
3,2020-03-18,Alberta,14566
4,2020-03-19,Alberta,17013


In [32]:
testcases_sorted['test_cases'] = 0
testcases_sorted['test_cases'] = testcases_sorted['test_cases'].astype(int)
testcases_sorted.dtypes
testcases_sorted.head()

,date,province,cumulative_testing,weekno,test_cases
id,,,,,
0,2020-03-15,Alberta,7108,11,0
1,2020-03-16,Alberta,10598,12,0
2,2020-03-17,Alberta,12355,12,0
3,2020-03-18,Alberta,14566,12,0
4,2020-03-19,Alberta,17013,12,0


In [33]:
prval = 0
prpro = 'test'
for index, row in testcases_sorted.iterrows():
    if row['province'] != prpro:
        prpro = row['province']
        prval = 0
    #print(row['province'], row['cumulative_testing'], prval, row['cumulative_testing'] - prval)
    testcases_sorted.loc[index, 'test_cases'] = (row['cumulative_testing'] - prval)
    prval = row['cumulative_testing']

In [34]:
testcases_sorted.head()

,date,province,cumulative_testing,weekno,test_cases
id,,,,,
0,2020-03-15,Alberta,7108,11,7108
1,2020-03-16,Alberta,10598,12,3490
2,2020-03-17,Alberta,12355,12,1757
3,2020-03-18,Alberta,14566,12,2211
4,2020-03-19,Alberta,17013,12,2447


In [35]:
# removing cumulative_testing column and grouping
testcases_sorted1 = testcases_sorted[['province', 'weekno', 'test_cases']]
testcases_grp = testcases_sorted1.groupby(['weekno','province']).sum()
testcases_grp.head()

test_cases
weekno province               
11     Alberta            7108
       BC                 6326
       Manitoba            400
       NL                  161
       Nova Scotia         418

In [36]:
recovered_df.head()
# recovered_df.columns

,date,province,cumulative_recovered
id,,,
36,2020-03-19,Alberta,2
37,2020-03-20,Alberta,3
38,2020-03-21,Alberta,3
39,2020-03-22,Alberta,3
40,2020-03-23,Alberta,3


In [37]:
recovered_df2 = recovered_df[['date', 'province', 'cumulative_recovered']]
recovered_df2['weekno'] = recovered_df2['date'].dt.week
recovered_df2.fillna('N/A', inplace=True)
recovered_sorted = recovered_df2.sort_values(["province","date"])
recovered_sorted['recovered'] = 0
recovered_sorted['recovered'] = recovered_sorted['recovered'].astype(int)
# recovered_sorted.dtypes
recovered_sorted.head()

,date,province,cumulative_recovered,weekno,recovered
id,,,,,
36,2020-03-19,Alberta,2,12,0
37,2020-03-20,Alberta,3,12,0
38,2020-03-21,Alberta,3,12,0
39,2020-03-22,Alberta,3,12,0
40,2020-03-23,Alberta,3,13,0


In [38]:
prval = 0
prpro = 'test'
for index, row in recovered_sorted.iterrows():
    if row['province'] != prpro:
        prpro = row['province']
        prval = 0
    #print(row['province'], row['cumulative_testing'], prval, row['cumulative_testing'] - prval)
    recovered_sorted.loc[index, 'recovered'] = (row['cumulative_recovered'] - prval)
    prval = row['cumulative_recovered']

In [39]:
recovered_sorted.head()

,date,province,cumulative_recovered,weekno,recovered
id,,,,,
36,2020-03-19,Alberta,2,12,2
37,2020-03-20,Alberta,3,12,1
38,2020-03-21,Alberta,3,12,0
39,2020-03-22,Alberta,3,12,0
40,2020-03-23,Alberta,3,13,0


In [40]:
# removing cumulative_recovered column and grouping
recovered_sorted1 = recovered_sorted[['province', 'weekno', 'recovered']]
recovered_grp = recovered_sorted1.groupby(['weekno','province']).sum()
recovered_grp = recovered_grp[recovered_grp['recovered'] > 0]
recovered_grp.head()

recovered
weekno province           
7      Ontario           1
8      BC                1
       Ontario           2
9      BC                3
10     Ontario           1

In [41]:
confirmed_cases_df.head()

,positive_cases,age,sex,province,date,travel_yn,travel_history_country,locally_acquired
id,,,,,,,,
0,1,50-59,Male,Ontario,2020-01-25,1,China,NaN
1,2,50-59,Female,Ontario,2020-01-27,1,China,NaN
2,1,40-49,Male,BC,2020-01-28,1,China,NaN
3,3,20-29,Female,Ontario,2020-01-31,1,China,NaN
4,2,50-59,Female,BC,2020-02-04,0,NaN,Close Contact


In [42]:
confirmed_df2 = confirmed_cases_df[['positive_cases', 'province', 'date']]
confirmed_df2['weekno'] = confirmed_df2['date'].dt.week
confirmed_grp = confirmed_df2.groupby(['weekno','province']).sum()
confirmed_grp.head()

positive_cases
weekno province                
4      Ontario                1
5      BC                     1
       Ontario                5
6      BC                     9
7      BC                     5

In [43]:
merged_df = pd.merge(confirmed_grp, recovered_grp, how="outer", left_on=["weekno","province"], right_on=["weekno","province"]).merge(testcases_grp, how="outer", left_on=["weekno","province"], right_on=["weekno","province"])
covid19_sum_canada = pd.merge(merged_df, mortality_grp, how="outer", left_on=["weekno","province"], right_on=["weekno","province"]) 
covid19_sum_canada.head()

positive_cases  recovered  test_cases  death_cases
weekno province                                                    
4      Ontario              1.0        NaN         NaN          NaN
5      BC                   1.0        NaN         NaN          NaN
       Ontario              5.0        NaN         NaN          NaN
6      BC                   9.0        NaN         NaN          NaN
7      BC                   5.0        NaN         NaN          NaN

In [44]:
covid19_sum_canada.reset_index(inplace=True)
covid19_sum_canada.index.name = 'id'
covid19_sum_canada.head()

,weekno,province,positive_cases,recovered,test_cases,death_cases
id,,,,,,
0,4,Ontario,1.0,NaN,NaN,NaN
1,5,BC,1.0,NaN,NaN,NaN
2,5,Ontario,5.0,NaN,NaN,NaN
3,6,BC,9.0,NaN,NaN,NaN
4,7,BC,5.0,NaN,NaN,NaN


In [45]:
# Trying to see all output tables fields that match with database tables before inserting...
confirmed_cases_df.head()

,positive_cases,age,sex,province,date,travel_yn,travel_history_country,locally_acquired
id,,,,,,,,
0,1,50-59,Male,Ontario,2020-01-25,1,China,NaN
1,2,50-59,Female,Ontario,2020-01-27,1,China,NaN
2,1,40-49,Male,BC,2020-01-28,1,China,NaN
3,3,20-29,Female,Ontario,2020-01-31,1,China,NaN
4,2,50-59,Female,BC,2020-02-04,0,NaN,Close Contact


In [46]:
mortality_df.head()

,province,date,age,sex
id,,,,
0,BC,2020-03-08,80-89,Male
1,Ontario,2020-03-11,70-79,Male
2,BC,2020-03-16,NaN,NaN
3,BC,2020-03-16,NaN,NaN
4,BC,2020-03-16,NaN,NaN


In [47]:
recovered_df.head()

,date,province,cumulative_recovered
id,,,
36,2020-03-19,Alberta,2
37,2020-03-20,Alberta,3
38,2020-03-21,Alberta,3
39,2020-03-22,Alberta,3
40,2020-03-23,Alberta,3


In [48]:
testcases_df.head()

,date,province,cumulative_testing
id,,,
0,2020-03-15,Alberta,7108
1,2020-03-16,Alberta,10598
2,2020-03-17,Alberta,12355
3,2020-03-18,Alberta,14566
4,2020-03-19,Alberta,17013


In [49]:
covid19_sum_canada.head()

,weekno,province,positive_cases,recovered,test_cases,death_cases
id,,,,,,
0,4,Ontario,1.0,NaN,NaN,NaN
1,5,BC,1.0,NaN,NaN,NaN
2,5,Ontario,5.0,NaN,NaN,NaN
3,6,BC,9.0,NaN,NaN,NaN
4,7,BC,5.0,NaN,NaN,NaN


### Create database connection

In [50]:
connection_string = "postgres:postgresus8@localhost:5432/covid19_db"
engine = create_engine(f'postgresql://{connection_string}')

In [51]:
# Confirm tables
engine.table_names()

['mortality', 'recovered', 'testcases', 'summary_data', 'confirmed_cases']

### Load DataFrames into database

In [52]:
mortality_df.to_sql(name='mortality', con=engine, if_exists='replace', index=True)

In [53]:
recovered_df.to_sql(name='recovered', con=engine, if_exists='replace', index=True)

In [54]:
testcases_df.to_sql(name='testcases', con=engine, if_exists='replace', index=True)

In [55]:
confirmed_cases_df.to_sql(name='confirmed_cases', con=engine, if_exists='replace', index=True)

In [56]:
covid19_sum_canada.to_sql(name='summary_data', con=engine, if_exists='replace', index=True)